# 1. Tips Piton + Errors

SUPER IMPORTANT: MOTIU TÍPIC DE RESULTAT INCORRECTE:
 - ES TREBALLA AMB ENTERS EN COMPTES DE FLOATS (posar x. en comptes de x al primer valor i s'arregla)
 - ES MODIFIQUEN VARIABLES PASSADES COM A PARÀMETRE DE LA FUNCIÓ (recomanable passar "AA" i fer "A = AA.copy()" al principi de la funció, per exemple). Això també pot passar dintre la funció (agafar u=A[:,0] i modificar u també modifica A)


In [ ]:
import numpy as np
import scipy
np.set_printoptions(precision=15, suppress=False)
A = np.array([[1,2],[-1,0]], dtype=np.float64) #o posar 1. al primer elem.

#per fer check:
np.linalg.norm(A)
np.linalg.det(A)
np.linalg.inv(A)
P, L, U = scipy.linalg.lu(A)
b = np.array([1,1]) # b random perque no doni compile error
x = np.linalg.solve(A, b)
vaps = np.linalg.eigvals(A) #.eig dona vaps + veps

idx = np.argsort(-np.abs(vaps)) # per ordenar array decreixentment (treure el - per creixent)
vaps = vaps[idx]

## Integrals i derivades

In [ ]:
from sympy import symbols, sin, cos, diff, integrate, pi, latex

# Creem la variable simbòlica x:
x = symbols('x')

# Funció:
f = sin(pi*x)*cos(pi*x)

# Calculem la derivada:
derivada = diff(f,x) # Podem calcular la derivada n-èssima mitjançant diff(f,x,n)

# Calculem la integral:
integral = integrate(f, x) # Podem calcular la integral entre a i b fent integrate(f,(x,a,b))

# Imprimim per pantalla el resultat:
print(f"La derivada de f(x) = {f} és {derivada}")
print(f"La integral de f(x) = {f} és {integral}")

La derivada de f(x) = sin(pi*x)*cos(pi*x) és -pi*sin(pi*x)**2 + pi*cos(pi*x)**2
La integral de f(x) = sin(pi*x)*cos(pi*x) és sin(pi*x)**2/(2*pi)


## Fórmules propagació errors

Per problemes numèrics on calculem $y=f(x)$ a partir d'una única dada $x$ tenim les següents fórmules aproximades de propagació de l'error:

$$ e_a(y) \approx f'(x)\cdot e_a(x),$$

$$ e_r(y) \approx \frac{f'(x)\cdot x}{f(x)}\,e_r(x),$$

on $e_a$ indica l'error absolut i $e_r$ l'error relatiu.  D'igual forma, tenim les fórmules aproximades de propagació de l'error maximal:

$$ \varepsilon_a(y) \approx |f'(x)|\cdot \varepsilon_a(x),$$

$$ \varepsilon_r(y) \approx \frac{|f'(x)\cdot x|}{|f(x)|}\,\varepsilon_r(x).$$


&nbsp;

Per problemes numèrics més generals, en els quals hem de calcular un resultat $y=g(x_1,x_2,...,x_n)$ a partir de les dades $x_1$, $x_2$, $\ldots$, $x_n$ tenim la generalitzacio de les fórmules aproximades de propagació de l'error:

$$ e_a(y) \approx \sum_{i=1}^n \frac{\partial g}{\partial x_i}\,(x_1,x_2,\ldots,x_n)\cdot e_a(x_i),$$

$$ e_r(y) \approx \frac{1}{g(x_1,x_2,\ldots,x_n)}\sum_{i=1}^n \frac{\partial g}{\partial x_i}\,(x_1,x_2,\ldots,x_n)\cdot x_i \cdot e_r(x_i).$$

I, d'igual manera, a partir de les fites $\varepsilon_a(x_i)$ dels errors absoluts $e_a(x_i)$ tenim la generalització de les fórmules aproximades de propagació de l'error maximal:

$$ \varepsilon_a(y) \approx \sum_{i=1}^n \left|\frac{\partial g}{\partial x_i}\,(x_1,x_2,\ldots,x_n)\right|\cdot \varepsilon_a(x_i),$$


$$ \varepsilon_r(y) \approx \frac{1}{|g(x_1,x_2,\ldots,x_n)|}\sum_{i=1}^n \left|\frac{\partial g}{\partial x_i}\,(x_1,x_2,\ldots,x_n)\right|\cdot |x_i| \cdot \varepsilon_r(x_i).$$

# 2. Resolució de sistemes lineals

## 2.1. Substitució enrere (trig superior) + Eliminació Gaussiana + endavant (trig inferior)

In [ ]:
import numpy as np
# resol Ux=b
def triU(U, b, tol=1e-10):
  n = len(U)
  x = np.array(b)
  for i in range(n-1, -1, -1):
    if abs(U[i,i]) < tol:
      raise ValueError("Element diagonal massa petit!")
    for j in range(i+1, n):
      x[i] -= U[i,j]*x[j]
    x[i] /= U[i,i]
  return x

# resol Ax=b (triangulitza A, i resol Ux=b)
def elimGauss(A,b,tol=1e-10):
  n = len(A)
  newA = A.copy()
  newB = b.copy()
  for k in range(n-1):
    for i in range(k+1, n):
      if abs(A[k,k]) < tol:
        print(k,A[k,k])
        raise ValueError("Element pivot massa petit!")
      mik = A[i,k] / A[k,k]
      newB[i] = b[i] - mik * b[k]
      for j in range(k, n):
        newA[i,j] = A[i,j] - mik * A[k,j]
    A = newA
    b = newB
  return triU(A, b, tol)

A = np.array([[1.0,0.5,0.33],[0.5,0.33,0.25],[0.33,0.25,0.20]])
b = np.array([1.0,0.0,0.0])
print(elimGauss(A,b))
print(np.linalg.solve(A, b))


[  55.55555555555388 -277.77777777776913  255.5555555555475 ]
[  55.55555555555482 -277.777777777774    255.55555555555208]


In [ ]:
# resol Lx=b, paràmetre opcional per reutilitzar M = L + U - Id
def triL(L, b, onesInDiag=False, tol=1e-10):
  n = len(L)
  x = np.array(b)
  for i in range(n):
    if not onesInDiag or abs(L[i,i]) < tol:
      raise ValueError("Element diagonal massa petit!")
    for j in range(i):
      x[i] -= L[i,j]*x[j]
    if not onesInDiag:
      x[i] /= L[i,i]
  return x

## 2.2. Factorització LU

In [ ]:
# A -> M (= L + U - Id) (LU Vanilla)
def factLU(A, tol = 1e-10):
  n = len(A)
  newA = A.copy()
  for k in range(n-1):
    for i in range(k+1, n):
      if abs(A[k,k]) < tol:
        raise ValueError("Element pivot massa petit!")
      mik = A[i,k] / A[k,k]
      for j in range(k+1, n):
        newA[i,j] = A[i,j] - mik * A[k,j]
      newA[i,k] = mik
    A = newA
  return A

A = np.array([[2.,-2,0],[100-2,2,0],[0,-1,3]])
print(factLU(A))

[[ 2.0e+00 -2.0e+00  0.0e+00]
 [ 4.9e+01  1.0e+02  0.0e+00]
 [ 0.0e+00 -1.0e-02  3.0e+00]]


In [ ]:
# A -> M, P amb pivotatge (M = L + U - Id)
def factLUpivot(AA, tol = 1e-10):
  A = AA.copy()
  n = len(A)
  P = np.array(range(n))

  #pivotatge parcial / esglaonat: canviar les 3 linies comentades (per defecte esglaonat)
  for k in range(n-1):
    i1 = k
    #s = abs(A[k,k])
    s = abs(A[k,k])/max(abs(A[k,k:]))
    for i in range(k+1, n):
      #if abs(A[i,k]) > s:
      if abs(A[i,k])/max(abs(A[i,k:])) > s:
        i1 = i
        #s = abs(A[i,k])
        s = abs(A[i,k])/max(abs(A[i,k:]))
    A[[k, i1]] = A[[i1, k]] # swap files
    P[i1], P[k] = P[k], P[i1] # actualitzar vector permutacions

    for i in range(k+1, n):
      if abs(A[k,k]) < tol:
        raise ValueError("Element pivot massa petit!")
      mik = A[i,k] / A[k,k]
      for j in range(k+1, n):
        A[i,j] = A[i,j] - mik * A[k,j]
      A[i,k] = mik
  return A, P

In [ ]:
# M -> L, U (separa L i U de M)
def separate(M):
  n = len(M)
  L = np.eye(n)
  U = np.zeros([n,n])
  for i in range(n):
    for j in range(n):
      if j >= i:
        U[i,j] = M[i,j]
      else:
        L[i,j] = M[i,j]
  return L, U

In [ ]:
# Exemple de solució (necessita triL i triU de 2.1)
A = np.array([[1.0,1,0], [2,0,2], [0,3,3]])
b = np.array([3, 8, 15])

M = factLU(A)
y = triL(M, b, True)
x = triU(M, y)

print(x)

# Per trobar la inversa d'una matriu
def invMat(A):
  n = len(A)
  M = factLU(A)
  invA = np.zeros([n,n])
  for i in range(n):
    b = np.zeros(n)
    b[i] = 1
    y = triL(M, b, True)
    x = triU(M, y)
    invA[:,i] = x
  return invA

A = np.array([
    [2, 3, 1],
    [4, 7, 3],
    [6, 18, 8]
])
print(invMat(A))
print(A@invMat(A), '\n', invMat(A)@A)

[1 2 3]
[[-0.25  0.75 -0.25]
 [ 1.75 -1.25  0.25]
 [-3.75  2.25 -0.25]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
# Igual que gauss (factLU) pero millor en casos. Retorna M = L + U - Id
def doolittle(A):
    n = len(A)
    M = np.zeros((n, n))
    for k in range(n):
        # Elements superiors (els de U)
        for j in range(k, n):
            suma = sum(M[k][m] * M[m][j] for m in range(k))
            M[k][j] = A[k][j] - suma
        # Elements inferiors (els de L)
        for i in range(k+1, n):
            suma = sum(M[i][m] * M[m][k] for m in range(k))
            M[i][k] = (A[i][k] - suma) / M[k][k]
    return M

## 2.3 Cholesky

In [ ]:
import numpy as np
from scipy.linalg import cholesky
np.set_printoptions(precision=15, suppress=False)

# A -> L
def Cholesky(A, tol = 1e-10):
  n = len(A)
  L = np.zeros([n,n])
  for k in range(n):
    if abs(A[k,k]) < tol:
      raise ValueError("Element pivot massa petit!")
    L[k,k] = A[k,k]
    for r in range(k):
      L[k,k] -= L[k,r]**2
    L[k,k] = L[k,k]**0.5
    for i in range(k+1,n):
      L[i,k] = A[i,k]
      for r in range(k):
        L[i,k] -= L[i,r]*L[k,r]
      L[i,k] /= L[k,k]
  return L
A = np.array([[5,1,1,1,1],
              [1,1,0,0,0],
              [1,0,1,0,0],
              [1,0,0,1,0],
              [1,0,0,0,1]],)
L = Cholesky(A)
print("A = \n ",L@np.transpose(L))
print("L = \n",L,"\n")
print("\n")
print("L = \n",cholesky(A,lower=True))


A = 
  [[ 5.000000000000001e+00  1.000000000000000e+00  1.000000000000000e+00
   1.000000000000000e+00  1.000000000000000e+00]
 [ 1.000000000000000e+00  9.999999999999999e-01 -6.297461701680791e-18
  -6.297461701680791e-18 -6.297461701680791e-18]
 [ 1.000000000000000e+00 -6.297461701680791e-18  9.999999999999999e-01
   1.213217268374993e-18  1.213217268374993e-18]
 [ 1.000000000000000e+00 -6.297461701680791e-18  1.213217268374993e-18
   1.000000000000000e+00 -1.991433824284152e-17]
 [ 1.000000000000000e+00 -6.297461701680791e-18  1.213217268374993e-18
  -1.991433824284152e-17  1.000000000000000e+00]]
L = 
 [[ 2.23606797749979   0.                 0.
   0.                 0.               ]
 [ 0.447213595499958  0.894427190999916  0.
   0.                 0.               ]
 [ 0.447213595499958 -0.223606797749979  0.866025403784439
   0.                 0.               ]
 [ 0.447213595499958 -0.223606797749979 -0.288675134594813
   0.816496580927726  0.               ]
 [ 0.44721359549

## 2.4. QR

In [ ]:
# A -> Q, R
def gramSchmidt(A):
  n = A.shape[1]
  Q = A.copy()
  R = np.zeros([n,n])
  for i in range(n):
    R[i,i] = np.linalg.norm(Q[:,i])
    Q[:,i] = Q[:,i]/R[i,i]
    for j in range(i+1,n):
      R[i,j] = Q[:,i].dot(Q[:,j])
      Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
  return Q, R

A = np.array([[1.,2],[2,2],[2,1]])
Q, R = gramSchmidt(A)
print(Q)
print(R)

[[ 0.333333333333333  0.808452083454443]
 [ 0.666666666666667  0.161690416690889]
 [ 0.666666666666667 -0.56591645841811 ]]
[[3.                2.666666666666667]
 [0.                1.374368541872554]]


## 2.5. Householder

In [ ]:
# retorna R, QtB (Ax=b <=> Rx=QtB), R triangular superior (cal eliminar el sobrant i quedar-se amb les mides pertinents)
def factHouseholder(AA, bb):
  #Molt important vigilar que AA i bb siguin de tipus float!
  A = AA.copy()
  b = bb.copy()
  m, n = np.shape(A)

  for k in range(n):
    u = A[k:, k].copy()
    normu = np.linalg.norm(u)
    if A[k,k] > 0:
      u[0] += normu
    else:
      u[0] -= normu
    uu = u/np.linalg.norm(u)

    for j in range(k,n):
      A[k:, j] = A[k:, j] - 2 * (uu * (uu.T @ A[k:, j]))
    b[k:] = b[k:] - 2 * (uu * (uu.T @ b[k:]))

  return A, b

# 3. Mètodes Iteratius

## 3.1. Jacobi

Convergent segur si A és estrictament diagonalment dominant (i altres casos)

In [ ]:
def norm(x):
  return sum(i*i for i in x)**0.5

def jacobi(A, b, x0, tol=1e-10, maxIter=100):
  n = len(A)
  x = x0.copy()

  for k in range(maxIter):
    xx = np.zeros(n)

    for i in range(n):
      suma = A[i,:]@x
      suma -= A[i][i] * x[i]
      xx[i] = 1/A[i][i] * (b[i] - suma)

    if norm(xx - x)/norm(xx) < tol:
      return xx, k+1

    x = xx.copy()

  return xx, -maxIter


## 3.2. Gauss-Seidel

Convergent segur si A és estrictament diagonalment dominant (i altres casos)

In [ ]:
def norm(x):
  return sum(i*i for i in x)**0.5

def gaussSeidel(A, b, x0, tol=1e-10, maxIter=100):
  n = len(A)
  x = x0.copy()

  for k in range(maxIter):
    xx = np.zeros(n)

    for i in range(n):
      for j in range(i):
        suma += A[i][j] * xx[j]
      for j in range(i+1, n):
        suma += A[i][j] * x[j]
      xx[i] = 1/A[i][i] * (b[i] - suma)

    if norm(xx - x)/norm(xx) < tol:
      return xx, k+1

    x = xx.copy()

  return xx, -maxIter

## 3.3. ω-relaxació successiva

ω∈(0,2) necessari perquè convergeixi el mètode.

In [ ]:
def norm(x):
  return sum(i*i for i in x)**0.5

# w entre 0 i 2 perque convergeixi, normalment a prop de 1 (com 1.01)
def wRelax(A, b, x0, w, tol=1e-10, maxIter=100):
  n = len(A)
  x = x0.copy()

  for k in range(maxIter):
    xx = np.zeros(n)

    for i in range(n):
      suma = 0
      for j in range(i):
        suma += A[i][j] * xx[j]
      for j in range(i+1, n):
        suma += A[i][j] * x[j]
      xx[i] = (1-w)*x[i] + w/A[i][i] * (b[i] - suma)

    if norm(xx - x)/norm(xx) < tol:
      return xx, k+1

    x = xx.copy()

  return xx, -maxIter

## 3.4. Mètode del gradient (+ conjugat)

La matriu ha de ser simètrica definida positiva (si no ho és, fer AtA)

In [ ]:
# MOLT LENT! Utilitzar gradient conjugat, no aquesta
def gradient(A, b, x0, tol=1e-10, maxIter=100):
  x = x0.copy()
  r = b - A@x

  for k in range(maxIter):
    alpha = r.dot(r)/r.dot(A@r) #alpha optim
    x += alpha*r #x(k+1)=x(k)+alpha(k)*rk
    r -= alpha*(A@r) #r(k+1)=rk-alpha(k)*A*rk
    if np.linalg.norm(r) < tol:
      return x, k+1
  return x, -maxIter

In [ ]:
# convergeix en <=n iteracions (?)
def gradientC1(A, b, x0, tol=1e-10, maxIter=100):
  x = x0.copy()
  r = b - A@x

  for k in range(maxIter):
    alpha = r.dot(r)/r.dot(A@r) #alpha optim
    x += alpha*r #x(k+1)=x(k)+alpha(k)*rk
    r -= alpha*(A@r) #r(k+1)=rk-alpha(k)*A*rk
    beta = r.dot(r)/r.dot(A@r)
    if np.linalg.norm(r) < tol:
      return x, k+1
  return x, -maxIter

def gradientC2(A, b, x0, tol=1e-10, maxIter=100):
  x = x0.copy()
  r = b - A@x
  p = r.copy()
  gamma = r.T@r
  for k in range(maxIter):
    y = A@p
    ak = gamma / (p.T@y)
    x += ak*p
    r = r - ak*y
    beta = (r.T@r) / gamma
    gamma = r.T@r
    p = r + beta * p
    if np.linalg.norm(r) < tol:
      return x, k+1

  return x, -maxIter

## 3.5. Jacobi per EDPs

In [ ]:
def norm_max(v):
  return max(abs(v.min()), abs(v.max()))

# interval [a,b], valors de frontera ca, cb, N valors, aplicant F
def solEstEq1D(a, b, ca, cb, N, F, maxIter, tol):
  h = (b-a)/N
  # generem el vector inicial de mida N+1, amb intervals uniformes
  umean = (ca+cb)/2
  u = umean*np.ones(N+1)
  u[0] = ca
  u[N] = cb

  h2F = np.zeros(N+1)
  for i in range(1,N):
    h2F[i] = F(i*h)*(h**2)

  for k in range(maxIter):
    # apliquem la funcio
    old_u = u.copy()
    u[1:N] = (u[:N-1] + u[2:] + h2F[1:-1])/2
    # comprovem tolerancia
    if norm_max(u - old_u) < tol:
      return u, k+1

  # no s'ha assolit la tolerancia: retornar nombre d'iteracions en negatiu
  return u, -maxIter

# 4. VAPs i VEPs

## 4.1. Mètode de la Potència

In [ ]:
#retorna el VAP mes gran en valor absolut de A. (z0 = np.random.rand(n))
def pot(A, z0, tol=1e-10, maxIter=100):
  rho = 0
  z = z0.copy()
  for k in range(maxIter):
    Az = A@z
    z = Az/np.linalg.norm(Az)
    rho_new = z.T@A@z

    if abs(rho_new - rho) < tol:
      return rho_new, z, k+1
    rho = rho_new

  return rho, z, -maxIter

A = np.array([[1,-3,3],[-1,1,1],[-1,-3,5]])
z = np.array([1,2,3])
print(pot(A,z))

(3.9999999999417946, array([7.07106781e-01, 4.11589929e-11, 7.07106781e-01]), 35)


In [ ]:
#necessita la majoria dels codis de 2.
def solve(A, b):
  M = factLU(A)
  y = triL(M, b, True)
  x = triU(M, y)
  return x

#retorna el VAP mes petit en valor absolut de A. (z0 = np.random.rand(n))
def potinv(A, z0, tol=1e-10, maxIter=100):
  rho = 0
  z = z0.copy()
  for k in range(maxIter):
    y = z / np.linalg.norm(z)
    z = solve(A, y)
    rho_new = y.T@z

    if abs(rho_new - rho) < tol:
      return 1/rho_new, z, k+1

    rho = rho_new

  return 1/rho, z, -maxIter

A = np.array([[1,-3,3],[-1,1,1],[-1,-3,5]])
z = np.array([1,2,3])
print(potinv(A,z))

NameError: name 'factLU' is not defined

In [ ]:
#retorna el VAP mes gran en valor absolut de A-qId. (z0 = np.random.rand(n))
def potdesp(A, z0, q, tol=1e-10, maxIter=100):
  M = A.copy() - q*np.eye(len(A))
  rho, z, iter = pot(M, z0, tol=tol, maxIter=maxIter)
  return rho + q , z0, iter

#retorna el VAP mes petit en valor absolut de A-qId. (z0 = np.random.rand(n))
def potinvdesp(A, z0, q, tol=1e-10, maxIter=100):
  M = A.copy() - q*np.eye(len(A))
  rho, z, iter = potinv(M, z0, tol=tol, maxIter=maxIter)
  return rho + q , z0, iter

# recomanable trobar el vap més gran i més petit,
# després el que estigui més a prop de (vapN+vap0)/2 = q,
# i anar iterant amb q de "punts mitjos" entre VAPs que tenen VAPs
# entremig encara (fer cheats i mirar els VAPs de veritat abans)


## 4.2. Mètode de Deflació

In [ ]:
import numpy as np
# necessita potRayleigh i potDespl de 4.1.
# retorna array de VAPs + array de VEPs
def deflacio(A, tol=1e-10, maxIter=10000):
  n = len(A)
  if n == 1:
    return [A[0,0]], np.array(1).reshape(1,1)
  z0 = np.random.rand(n) # z0 random en comptes de tot uns...
  rho, z, iter = pot(A, z0, tol=tol, maxIter=maxIter)
  if iter == -maxIter: # si no convergeix, provar q=1
    rho, z, iter = potdesp(A, z0, 1, tol=tol, maxIter=maxIter)

  u = z.copy()
  if u[0] >= 0:
    u[0] += 1.
  else:
    u[0] -= 1.
  u = np.reshape(u,(n,1))
  H = np.eye(n) - 2*u.dot(u.T)/(u.T.dot(u))
  A1 = H@A@H
  lamb = A1[0,0]
  b = A1[0,1:]

  vaps1, veps1 = deflacio(A1[1:,1:], tol=tol, maxIter=maxIter)

  vaps = np.zeros(n)
  veps = np.zeros([n,n])
  vaps[0] = lamb # hauria de ser = rho?
  veps[0] = z

  # converteix tots els veps de A1 "petita" a veps de A
  for i in range(n-1):
    alpha = b.dot(veps1[i])/(vaps1[i] - lamb)
    w2 = alpha * np.ones(n)
    w2[1:] = veps1[i]
    vaps[i+1] = vaps1[i]
    veps[i+1] = H@w2

  return vaps, veps

A = np.array([[1.,2,3],[2,0,4],[3,4,1]])
# comparacio numpy vs funcio:
vaps, veps = np.linalg.eig(A)
ind = np.argsort(vaps)
vaps1,veps1 = deflacio(A)
ind1 = np.argsort(vaps1)
for i in range(len(A)):
    print(f"VAP {i+1}: {vaps[ind[i]]} vs {vaps1[ind1[i]]}")
    print("VEP numpy   :", veps[:,ind[i]])
    print("VEP deflacio:", veps1[ind1[i]]/np.linalg.norm(veps1[ind1[i]]),"\n")
    #print("vep_i @ vepdefla_i= +1 o -1:",veps1[ind1[i]]@veps[:,ind[i]]/np.linalg.norm(veps1[:,ind1[i]]),"\n") # si dona ±1 esta be


## 4.3. Mètode de Jacobi

In [ ]:
# torna array de VAPs + array de VEPs (+ num rotacions/iteracions)
# la matriu A de input HA DE SER SIMETRICA
def jacobi(AA, tol=1e-10, maxRot=100):
  A = AA.copy()
  n = len(A)
  Q = np.eye(n)
  for k in range(maxRot+1):
    AnoDiag = A.copy()
    np.fill_diagonal(AnoDiag, 0)
    argmax = np.argmax(np.abs(AnoDiag))
    p, q = argmax//n, argmax%n

    if abs(A[p,q]) < tol:
      return np.diag(A), Q, k+1

    eta = (A[p,p] - A[q,q])/A[p,q]/2
    t = -eta + np.sign(eta) * (eta**2+1)**0.5
    if eta == 0:
      t = 1
    c = 1/(1 + t**2)**0.5
    s = t*c

    Anew = A.copy()

    Anew[p,:] = A[p,:]*c + A[q,:]*s
    Anew[q,:] = -A[p,:]*s + A[q,:]*c
    Anew[:,p] = A[:,p]*c + A[:,q]*s
    Anew[:,q] = -A[:,p]*s + A[:,q]*c

    Anew[p,p] = A[p,p]*c**2 + 2*A[p,q]*c*s + A[q,q]*s**2
    Anew[p,q] = Anew[q,p] = 0
    Anew[q,q] = A[p,p]*s**2 - 2*A[p,q]*c*s + A[q,q]*c**2

    A = Anew

    Qi = np.eye(n)
    Qi[p,p] = Qi[q,q] = c
    Qi[p,q] = -s
    Qi[q,p] = s
    Q = Q@Qi
  return np.diag(A), Q, -maxRot

A = np.array([[2.0,-1.0,0.0],[-1.0,2.0,-1.0],[0.0,-1.0,2.0]])
# comparacio numpy vs funcio:
vaps, veps = np.linalg.eig(A)
ind = np.argsort(vaps)
vaps1,veps1,rot = jacobi(A)
ind1 = np.argsort(vaps1)
for i in range(len(A)):
    print(f"VAP {i+1}: {vaps[ind[i]]} vs {vaps1[ind1[i]]}")
    print("VEP numpy   :", veps[:,ind[i]])
    print("VEP jacobi:", veps1[:,ind1[i]]/np.linalg.norm(veps1[:,ind1[i]]),"\n")
    #print("vep_i @ vepdefla_i= +1 o -1:",veps1[ind1[i]]@veps[:,ind[i]]/np.linalg.norm(veps1[:,ind1[i]]),"\n") # si dona ±1 esta be

## 4.a. Discs de Gerschgorin

In [ ]:
## Fem la intersecció entre la unió de discs per files i la unió de discs per columnes
#  per saber on estan localitzats els valors propis:
import numpy as np
import matplotlib.pyplot as plt

# Aquesta funció ens retorna els centres i radis dels discos
# de Gerschgorin per files.
def discsGerschgorin(A):
    n, m = A.shape
    if n != m:
        raise Exception("Error! La matriu no és quadrada")
    c = np.diag(A) # Calculem els centres
    r = np.linalg.norm(A, axis =1, ord=1) - np.absolute(c) # Calculem els radis
    return c, r

## Procediment general:
# Matriu A "semi"-random:
A = A = np.array([[0,0,2,0,3],
              [3,2,2,2,3],
              [1,1,3,1,1],
              [1,1,2,3,2],
              [3,3,1,0,1]],dtype=np.float64)

# Calculem els vaps i els veps:
vaps, veps = np.linalg.eig(A)
# Disc de Gerschgorin per files i columnes
c,r = discsGerschgorin(A)
c2,r2 = discsGerschgorin(A.T)
# Centres i radis:
#print(c,r)
n = len(A)
## Dibuixem:
fig, ax = plt.subplots()
ax.set_aspect('equal')  # Fem l'escala dels eixos iguals
ax.grid(True) # Afegim el grid
# Anem dibuixant cada disc (per files):
for i in range(n):
    circle = plt.Circle((np.real(c[i]), np.imag(c[i])), r[i], color='blue', fill=True, alpha=0.3)
    ax.add_patch(circle)
# Anem dibuixant cada disc (per columnes):
for i in range(n):
    circle = plt.Circle((np.real(c2[i]), np.imag(c2[i])), r2[i], color='orange', fill=True, alpha=0.3)
    ax.add_patch(circle)
# Dibuixem els VAPS:
plt.scatter(np.real(vaps),np.imag(vaps),color="black", s=5)
# Limits dels eixos (CANVIAR PARAMETRES SI NO ES VEU EL GRAFIC SENCER!):
plt.xlim(-20, 20)
plt.ylim(-20, 20)
# Labels dels eixos:
plt.xlabel('Part real')
plt.ylabel('Part imaginària')
plt.show()

## 4.b. Exemple busca de VAPs amb desplaçaments

In [ ]:
# (els desplaçaments i inversos es fan sabent mes o menys on son els VAPs, utilitzant els discos)
# Matriu de l'exercici
A = np.array([[7,-1,1,1,-1],[1,-5,0,0,-1],[0,0,14,-1,1],[-1,-1,0,-12,1],[0,1,1,0,0]],dtype = np.float64)
vaps, veps = np.linalg.eig(A)
print("Els VAPS són:",vaps)

z0 = np.random.rand(5)
vap1, vep1, iter1 = pot(A,z0,tol=1.e-12,maxIter = 300)
print("Vap 1:",vap1,"(en",iter1,"iteracions)")

# Podem millorar la convergència desplaçant el VAP per millorar la convergència
z0 = np.random.rand(5)
vap1, vep1, iter1 = potdesp(A,z0,-2.5,tol=1.e-12,maxIter = 300)
print("Vap 1:",vap1,"(en",iter1,"iteracions)")

z0 = np.random.rand(5)
vap2, vep2, iter2 = potinvdesp(A,z0,6,tol=1.e-12,maxIter = 300)
print("Vap 2:",vap2,"(en",iter2,"iteracions)")


z0 = np.random.rand(5)
vap3, vep3, iter3 = potdesp(A,z0,5,tol=1.e-12,maxIter = 300)
print("Vap 3:",vap3,"(en",iter3,"iteracions)")


z0 = np.random.rand(5)
vap4, vep4, iter4 = potinvdesp(A,z0,-4,tol=1.e-12,maxIter = 300)
print("Vap 4:",vap4,"(en",iter4,"iteracions)")


z0 = np.random.rand(5)
vap5, vep5, iter5 = potinv(A,z0,maxIter = 300)
print("Vap 5:",vap5,"(en",iter5,"iteracions)")


# Construixo la matriu que conté els VEPS i la seva inversa:
V = np.array([vep1,vep2,vep3,vep4,vep5]).T
invV = np.linalg.inv(V)
# Matriu diagonal amb els VAPs:
D = np.diag([vap1,vap2,vap3,vap4,vap5])

print("||A-VDV^-1|| =",np.linalg.norm(V@D@invV-A))